<a href="https://colab.research.google.com/github/BhardwajG572/21_day_21_projects/blob/main/Project_15_Assignment_language_to_SQL_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!curl "https://api.mockaroo.com/api/0009a690?count=1000&key=952f7f00" > "Students.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 86212    0 86212    0     0  40248      0 --:--:--  0:00:02 --:--:-- 40267


In [24]:
!curl "https://api.mockaroo.com/api/8c44c310?count=1000&key=952f7f00" > "Courses.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 47822    0 47822    0     0  49923      0 --:--:-- --:--:-- --:--:-- 49918


In [25]:
!curl "https://api.mockaroo.com/api/d10d9fd0?count=1000&key=952f7f00" > "Enrollments.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 36178    0 36178    0     0  12203      0 --:--:--  0:00:02 --:--:-- 12201


In [26]:
import sqlite3
import pandas as pd
import os

In [27]:
db_name = 'University.db'
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

In [49]:
import sqlite3
import pandas as pd
import os

students_schema = """
CREATE TABLE IF NOT EXISTS Students (
    student_id INT PRIMARY KEY,
    enrollment_date DATE NOT NULL,
    first_name VARCHAR(100) NOT NULL,
    last_name VARCHAR(100) NOT NULL,
    major VARCHAR(100),
    email VARCHAR(255) NOT NULL UNIQUE,
    phone_number VARCHAR(20),
    date_of_birth DATE
);
"""

courses_schema = """
CREATE TABLE IF NOT EXISTS Courses (
    course_id INT PRIMARY KEY,
    course_title VARCHAR(255) NOT NULL,
    course_code VARCHAR(20) NOT NULL UNIQUE,
    department VARCHAR(100) NOT NULL,
    credits INT NOT NULL,
    professor_name VARCHAR(255)
);
"""

enrollments_schema = """
CREATE TABLE IF NOT EXISTS Enrollments (
    enrollment_id INT PRIMARY KEY,
    student_id INT NOT NULL,
    course_id INT NOT NULL,
    enrollment_date DATETIME NOT NULL,
    semester VARCHAR(50) NOT NULL,
    grade VARCHAR(2),
    FOREIGN KEY (student_id) REFERENCES Students(student_id),
    FOREIGN KEY (course_id) REFERENCES Courses(course_id)
);
"""

# --- Database setup ---
db_name = 'UniversityDB.db' # Using the name from your file list
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

conn = None  # Initialize connection to None

try:
    # Establish a connection to the SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"Database '{db_name}' created and connected successfully. ✅")

    cursor.execute(students_schema)
    cursor.execute(courses_schema)
    cursor.execute(enrollments_schema)
    print("Tables 'Students', 'Courses', and 'Enrollments' created successfully.")

    # --- Load data from CSV files into the tables using pandas ---
    # This dictionary correctly maps your CSV files to the newly created tables
    csv_to_table_map = {
        '/content/Students.csv': 'Students',
        '/content/Courses.csv': 'Courses',
        '/content/Enrollments.csv': 'Enrollments'
        # I've assumed you renamed 'Enrollments Dataset.csv' to 'Enrollments.csv' for simplicity
    }

    # This dictionary for enforcing data types should also be defined
    # You would need to create this based on your CSV columns
    COLUMN_DATA_TYPES = {
        'Students': { 'student_id': 'int64', 'enrollment_date': 'datetime64[ns]', 'date_of_birth': 'datetime64[ns]' },
        'Courses': { 'course_id': 'int64', 'credits': 'int64' },
        'Enrollments': { 'enrollment_id': 'int64', 'student_id': 'int64', 'course_id': 'int64', 'enrollment_date': 'datetime64[ns]' }
    }


    for csv_file, table_name in csv_to_table_map.items():
        if os.path.exists(csv_file):
            print(f"\nProcessing '{csv_file}' for table '{table_name}'...")
            df = pd.read_csv(csv_file)

            if table_name in COLUMN_DATA_TYPES:
                expected_schema = COLUMN_DATA_TYPES[table_name]
                for col, dtype in expected_schema.items():
                    if 'datetime' in dtype:
                        df[col] = pd.to_datetime(df[col], errors='coerce')
                    else:
                        try:
                            df[col] = df[col].astype(dtype)
                        except (ValueError, TypeError):
                            pass

            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"  -> Data from '{csv_file}' loaded into '{table_name}' table successfully.")
        else:
            print(f"Warning: '{csv_file}' not found. Skipping data load for '{table_name}'.")

    conn.commit()
    print("\nData committed to the database successfully. 🎉")

except sqlite3.Error as e:
    print(f"Database error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    if conn:
        conn.close()
        print("Database connection closed.")

Removed existing database 'UniversityDB.db'.
Database 'UniversityDB.db' created and connected successfully. ✅
Tables 'Students', 'Courses', and 'Enrollments' created successfully.

Processing '/content/Students.csv' for table 'Students'...
  -> Data from '/content/Students.csv' loaded into 'Students' table successfully.

Processing '/content/Courses.csv' for table 'Courses'...
  -> Data from '/content/Courses.csv' loaded into 'Courses' table successfully.

Processing '/content/Enrollments.csv' for table 'Enrollments'...
  -> Data from '/content/Enrollments.csv' loaded into 'Enrollments' table successfully.

Data committed to the database successfully. 🎉
Database connection closed.


In [30]:
## Setup Google AI Studio API Key :

In [31]:
!pip install google-genai

In [32]:
import google.generativeai as genai
from google.colab import userdata

In [33]:
genai_model = genai.GenerativeModel('gemini-2.5-flash')


In [34]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [35]:
genai_model

genai.GenerativeModel(
    model_name='models/gemini-2.5-flash',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

In [36]:
prompt = """
### **ROLE**

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### **CONTEXT**

You are the core translation engine for a University  intelligence dashboard. This tool allows non-technical students to query the university's  database using natural language. The database dialect is always **SQLite**. Your responses will be executed directly on the database.
(Your main task is to help the students who is asking for the query  to you in natural language and your task is to give the desired output(put in dataframe) along with sqlite codes)

The database consists of the following three tables:

-- Creates the Students table
CREATE TABLE Students (
    student_id INT PRIMARY KEY AUTO_INCREMENT, -- Use SERIAL for PostgreSQL
    enrollment_date DATE NOT NULL,
    first_name VARCHAR(100) NOT NULL,
    last_name VARCHAR(100) NOT NULL,
    major VARCHAR(100),
    email VARCHAR(255) NOT NULL UNIQUE,
    phone_number VARCHAR(20),
    date_of_birth DATE
);

-- Creates the Courses table
CREATE TABLE Courses (
    course_id INT PRIMARY KEY AUTO_INCREMENT, -- Use SERIAL for PostgreSQL
    course_title VARCHAR(255) NOT NULL,
    course_code VARCHAR(20) NOT NULL,
    department VARCHAR(100) NOT NULL,
    credits INT NOT NULL,
    professor_name VARCHAR(255)
);

-- Creates the Enrollments table and links it to Students and Courses
CREATE TABLE Enrollments (
    enrollment_id INT PRIMARY KEY AUTO_INCREMENT, -- Use SERIAL for PostgreSQL
    student_id INT NOT NULL,
    course_id INT NOT NULL,
    enrollment_date DATETIME NOT NULL,
    semester VARCHAR(50) NOT NULL,
    grade VARCHAR(2),
    FOREIGN KEY (student_id) REFERENCES Students(student_id),
    FOREIGN KEY (course_id) REFERENCES Courses(course_id)
);


Of course! Here is a world-class, structured prompt engineered for the university database schema, following your detailed instructions.

##ROLE##
You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries along with output in (desired dataframe) based on the provided university database schema.

##CONTEXT##
You are the core translation engine for a university's student information system dashboard. This tool allows non-technical faculty/students and administrators to query the database using natural language. The database dialect is always SQLite.

The database consists of the following three tables which are linked by foreign keys in the Enrollments table:

Students table:

SQL

CREATE TABLE Students (
    student_id INT PRIMARY KEY,
    enrollment_date DATE NOT NULL,
    first_name VARCHAR(100) NOT NULL,
    last_name VARCHAR(100) NOT NULL,
    major VARCHAR(100),
    email VARCHAR(255) NOT NULL UNIQUE,
    phone_number VARCHAR(20),
    date_of_birth DATE
);
Courses table:

SQL

CREATE TABLE Courses (
    course_id INT PRIMARY KEY,
    course_title VARCHAR(255) NOT NULL,
    course_code VARCHAR(20) NOT NULL UNIQUE,
    department VARCHAR(100) NOT NULL,
    credits INT NOT NULL,
    professor_name VARCHAR(255)
);
Enrollments table:

SQL

CREATE TABLE Enrollments (
    enrollment_id INT PRIMARY KEY,
    student_id INT NOT NULL,
    course_id INT NOT NULL,
    enrollment_date DATETIME NOT NULL,
    semester VARCHAR(50) NOT NULL,
    grade VARCHAR(2),
    FOREIGN KEY (student_id) REFERENCES Students(student_id),
    FOREIGN KEY (course_id) REFERENCES Courses(course_id)
);

-----

##TASK##
Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query. Follow these steps meticulously:

Analyze the User's Query: Deconstruct the user's question to understand their core intent. Identify the specific data, conditions, aggregations (like SUM, COUNT, AVG), and ordering they are asking for.

Map to the Schema: Map the entities from the user's query to the appropriate tables (Students, Courses, Enrollments) and columns. Determine the necessary JOIN operations.

Construct the SQLite Query: Write a clean and efficient SELECT statement that is syntactically correct for SQLite. Ensure all table and column names are accurate.

Handle Ambiguity: If the user's query is vague or lacks necessary information, formulate a specific, targeted question to ask the user for clarification.

## CONSTRAINTS
Read-Only Operations: You must ONLY generate SELECT queries. Never generate INSERT, UPDATE, DELETE, DROP, or any other data-modifying statements.

Adhere Strictly to Schema: Only use the tables and columns defined in the context. Do not invent or assume the existence of any other tables or columns.

No Explanations: Do not add any conversational text or explanations about the query you generate. Your output must strictly follow the specified format.

Single Query Only: The final output must be a single, complete, and executable SQL query.

Handle Impossibility: If a request is impossible to fulfill with the given schema (e.g., "Which professor has the highest salary?"), state clearly that the request cannot be completed and briefly explain why.




## EXAMPLES
**Example 1: Simple Lookup

**User Query: "Show me all courses in the History department"

**Expected Output

JSON

{
  "status": "success",
  "response": "SELECT * FROM Courses WHERE department = 'History';"
}

**Example 2: Complex Join and Aggregation

**User Query: "What is the name of the student with the most course enrollments?"

**Expected Output:

JSON

{
  "status": "success",
  "response": "SELECT T1.first_name, T1.last_name FROM Students AS T1 JOIN Enrollments AS T2 ON T1.student_id = T2.student_id GROUP BY T1.student_id ORDER BY COUNT(T2.course_id) DESC LIMIT 1;"
}

**Example 3: Ambiguous Query

**User Query: "Show me new students"

**Expected Output:

JSON

{
  "status": "clarification_needed",
  "response": "Could you please define what 'new' means? For example, 'enrolled this semester' or 'enrolled in 2025'."
}

**Example 4: Impossible Query

**User Query: "Which professor has the highest salary?"

**Expected Output could be :



## OUTPUT FORMAT
Your final response must be a single JSON object with two keys:

"status": A string with one of three possible values: "success", "clarification_needed", or "error".

"response":

If status is "success", this will be a string containing the complete SQLite query.

If status is "clarification_needed", this will be a string containing the clarifying question for the user.

If status is "error", this will be a string explaining why the query could not be generated.

"""


In [37]:
import json
import re
from typing import Dict, Any


def get_sql_query(genai_client: Any, prompt: str, user_query: str) -> Dict[str, str]:
    """
    Generates an SQL query from a natural language user query using a GenAI model.

    This function constructs a detailed prompt, sends it to the specified Gemini model,
    prints token usage, and safely parses the JSON response.

    Args:
        genai_client: An initialized generative model client (e.g., from google.generativeai).
        prompt: The master prompt string containing the role, context, task, etc.
        user_query: The natural language question from the end-user.

    Returns:
        A dictionary containing the 'status' and 'response' from the model.
        Returns a dictionary with an error status if the API call or JSON parsing fails.
    """
    contents = f"""
    {prompt}

    Here's the user query in english you need to work on:
    {user_query}
    """

    try:
        # Call the generative model API
        response = genai_client.generate_content(contents=contents)

        # Print the token usage metadata for debugging and cost monitoring
        usage_metadata = response.usage_metadata
        print(f"Input Token Count: {usage_metadata.prompt_token_count}")
        print(f"Output Token Count: {usage_metadata.candidates_token_count}")
        print(f"Total Token Count: {usage_metadata.total_token_count}")

        # The model is instructed to return a JSON object, sometimes within a markdown block.
        # This regex safely extracts the content inside the ```json ... ``` block.
        json_match = re.search(r"```json\n(.*?)\n```", response.text, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
        else:
            # Fallback if the model doesn't use a markdown block
            json_str = response.text

        # Safely load the JSON string into a Python dictionary
        output = json.loads(json_str)
        return output

    except json.JSONDecodeError:
        print(f"Error: Failed to decode JSON from the model's response.\nResponse Text:\n{response.text}")
        return {
            "status": "error",
            "response": "The model returned a malformed response that could not be parsed as JSON."
        }
    except Exception as e:
        print(f"An unexpected error occurred during the API call: {e}")
        return {
            "status": "error",
            "response": f"An API or network error occurred: {e}"
        }

In [38]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='UniversityDB.db'):

    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the query
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Get column names from the cursor description
        columns = [description[0] for description in cursor.description]

        # Format results as a dataframe for easier use
        results_as_dict = [dict(zip(columns, row)) for row in results]
        results_df = pd.DataFrame(results_as_dict)

        print("Query executed successfully.")
        return results_df

    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()

In [40]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query(genai_client, prompt, user_query)
  if output['status'] == 'success':
    results = execute_query(output['response'])
    return results
  return output

In [41]:
text2sql(genai_model, prompt, "How many orders were placed using 'Credit Card' as the payment method?")

Input Token Count: 1858
Output Token Count: 55
Total Token Count: 1993


{'status': 'error',
 'response': "I cannot answer this question as the database does not contain information about 'orders' or 'payment methods'. The available tables are Students, Courses, and Enrollments."}

In [42]:
text2sql(genai_model, prompt, "what are the distinct professor name ? ")

Input Token Count: 1851
Output Token Count: 37
Total Token Count: 1938

Executing query on 'UniversityDB.db':
SELECT DISTINCT professor_name FROM Courses WHERE professor_name IS NOT NULL;
Query executed successfully.


,professor_name
0,Clerissa Dalgety
1,Bay Fardoe
2,Samara Stookes
3,Audie Vigours
4,Zuzana Dutson
...,...
995,Candi Becerro
996,Murdoch Bickerdicke
997,Nikoletta Ardley
998,Darlene Tieman


In [44]:
text2sql(genai_model, prompt, "Show me all enrollments for the 'Fall 2022' semester")

Input Token Count: 1859
Output Token Count: 38
Total Token Count: 1984

Executing query on 'UniversityDB.db':
SELECT * FROM Enrollments WHERE semester = 'Fall 2022';
Query executed successfully.


,enrollment_id,student_id,course_id,enrollment_date,semester,grade
0,5,225,89,2025-05-21 00:00:00,Fall 2022,A
1,36,525,84,2025-04-10 00:00:00,Fall 2022,D
2,54,89,208,2025-02-19 00:00:00,Fall 2022,F
3,60,757,372,2025-01-09 00:00:00,Fall 2022,F
4,63,496,386,2024-11-18 00:00:00,Fall 2022,D
...,...,...,...,...,...,...
78,936,906,999,2024-11-04 00:00:00,Fall 2022,C
79,950,837,601,2025-09-27 00:00:00,Fall 2022,B
80,967,795,144,2024-12-02 00:00:00,Fall 2022,A
81,968,652,369,2025-08-05 00:00:00,Fall 2022,C


In [46]:
text2sql(genai_model, prompt,  "For each major, rank students by their enrollment date, showing the earliest enrolled students first.")

Input Token Count: 1861
Output Token Count: 79
Total Token Count: 2093

Executing query on 'UniversityDB.db':
SELECT s.first_name, s.last_name, s.major, s.enrollment_date, ROW_NUMBER() OVER (PARTITION BY s.major ORDER BY s.enrollment_date ASC) as rank_in_major FROM Students AS s;
Query executed successfully.


,first_name,last_name,major,enrollment_date,rank_in_major
0,Sammy,McCormack,Art,2024-10-03 00:00:00,1
1,Suzanna,Ivankin,Art,2024-10-07 00:00:00,2
2,Rosemary,Nawton,Art,2024-10-09 00:00:00,3
3,Davidson,Crimin,Art,2024-10-11 00:00:00,4
4,Mack,Haile,Art,2024-10-17 00:00:00,5
...,...,...,...,...,...
995,Jyoti,Munford,Sociology,2025-08-29 00:00:00,78
996,Ulrick,Savil,Sociology,2025-09-05 00:00:00,79
997,Kimball,Bonnaire,Sociology,2025-09-16 00:00:00,80
998,Aldon,Chatburn,Sociology,2025-09-20 00:00:00,81


In [47]:
text2sql(genai_model, prompt,  "Which departments have an average course credit value greater than 3?")

Input Token Count: 1856
Output Token Count: 38
Total Token Count: 1992

Executing query on 'UniversityDB.db':
SELECT department FROM Courses GROUP BY department HAVING AVG(credits) > 3;
Query executed successfully.


,department
0,1000
1,103
2,107
3,115
4,122
...,...
240,986
241,992
242,993
243,997


In [48]:
"Show me the total number of student enrollments for each month of the year."
text2sql(genai_model, prompt, "Show me the total number of student enrollments for each month of the year."
 )

Input Token Count: 1859
Output Token Count: 63
Total Token Count: 2096

Executing query on 'UniversityDB.db':
SELECT strftime('%m', enrollment_date) AS enrollment_month, COUNT(enrollment_id) AS total_enrollments FROM Enrollments GROUP BY enrollment_month ORDER BY enrollment_month;
Query executed successfully.


,enrollment_month,total_enrollments
0,01,88
1,02,72
2,03,100
3,04,85
4,05,85
5,06,83
6,07,78
7,08,90
8,09,79
9,10,72
